<a href="https://colab.research.google.com/github/pycriador/PySendEmail/blob/main/pySendEmail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python SendEmail (Gmail version)

Um script simples para enviar e-mails com referências uma base no Excel, anexando uma mensagem em HTML (se desejar), colocando anexos (se desejar) e utilizando o próprio SMTP do Google (gmail). 

In [ ]:
!pip install ninja2

In [12]:
import smtplib
from email import encoders
#Python 2
#from email.MIMEMultipart import MIMEMultipart
#from email.MIMEText import MIMEText
#from email.MIMEBase import MIMEBase
#Python 3
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
#Modificar informações do HTML
from jinja2 import Template, Environment, FileSystemLoader
#Modificar informação do DOCX
#from docxtpl import DocxTemplate

Informe o arquivo XLSX (arquivo Excel) com os dados que você deseja preencher no template

In [ ]:
from google.colab import files
import os
import shutil

print('Removendo arquivos de exemplo')
try: shutil.rmtree('sample_data')
except: pass

print('Escolha o arquivo XLSX')
ap_up = files.upload()

#print('Limpando bases antigas')
#try:
#  os.remove('base.xlsx')
#except Exception as e:
#  print(f'Erro: {e}')

#for upload in ap_up:
#  os.rename(upload, "base.xlsx")

In [ ]:
from google.colab import files

print('Escolha o arquivo HTML')
ap_up = files.upload()

print('Criando pasta de templates')
try:
  !mkdir templates
except Exception as e:
  print(f'Erro: {e}')

print('Movendo arquivos HTML para pasta de template')
!mv *.html templates

Se desejar anexar arquivos no seu e-mail, realizar o upload por aqui

In [ ]:
from google.colab import files

print("Carregar anexos")
ap_up = files.upload()

Iniciando o programa

In [36]:
import pandas as pd
from jinja2 import Template, Environment, FileSystemLoader

class SendEmail():
  def __init__(self, fromaddr=None, subject=None):
    #Configura variáveis da Classe
    self.fromaddr = fromaddr
    self.subject = subject

    #Cria a mensagem como MIMEMultipart
    self.msg = MIMEMultipart()
    #Já adiciona o remetente e o assunto
    self.msg['From'] = fromaddr
    self.msg['Subject'] = subject

  def read_base(self, file=None, sheet=None):
    #Abrir planilha Excel em XLSX
    xlsx = pd.ExcelFile(file)
    if sheet:
      return pd.read_excel(xlsx, sheet)
    else:
      return pd.read_excel(xlsx)

  def attachfile(self, filename=None):
    #Abrir o anexo
    attachment = open(filename, "rb")
    #Converter o anexo
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    #Adicionar o header
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    #Adicionar o anexo na mensagem principal
    self.msg.attach(part)
    #Fechar o arquivo
    attachment.close()

  def read_template(self, templates='templates', page=None, encoding='ISO-8859-1'):
    # Carregar diretório de templates
    file_loader = FileSystemLoader(templates, encoding=encoding)
    env = Environment(loader=file_loader)
    # Carregar o template
    self.template = env.get_template(page)

  def add_mensage_html(self, htmlfile=None, encoding='UTF-8', data=None):
    app.read_template(templates='templates', page=htmlfile, encoding=encoding)
    html = app.template.render(data)
    self.msg.attach(MIMEText(html,'html','UTF-8'))

  def smtp_config(self, smtpserver=None, smtpport=465, login=None, password=None):
    #Configurações do Servidor Gmail
    self.server = smtplib.SMTP_SSL(smtpserver, int(smtpport))
    self.server.ehlo()
    #Informar usuário e senha para autenticação
    self.server.login(login, password)

  def send_email(self, toaddr=None):
    try:
      #Adicionar o destinatário
      self.msg['To'] = toaddr
      #Converter a mensagem em string
      text = self.msg.as_string()
      self.server.sendmail(self.fromaddr, [toaddr], text)
      #self.server.quit()
    except Exception as e:
      print(e)


Agora você pode personalizar os dados de envio. Os comentários estão explicando cada parte do código, qualquer dúvida é só me chamar!

In [37]:
#Iniciar o programa informando quem vai mandar o e-mail e qual é o assunto
app = SendEmail(fromaddr='@gmail.com', subject='Muito obrigado!')

#Dados enviados para substituição no template HTML
data = {'NOME': 'Sr. Vinicius', 'CONTATO': '@gmail.com'}
app.add_mensage_html(htmlfile='mensagem.html', encoding='UTF-8', data=data)

#Para adicionar anexo
app.attachfile(filename='base.xlsx')
# Adicionar o segundo anexo (se quiser)
#app.attachfile(filename='templates/mensagem.html')

# Para autenticar no SMTP do Google, você precisa criar uma aplicação, maiores 
# detalhes na documentação (Inglês)
# Documentação: https://support.google.com/mail/answer/185833?hl=en
# Resumindo, você precisa acessar a URL:
# https://myaccount.google.com/
# Ir em:
# Segurança -> Como fazer login no Google (Senhas de App) -> Login -> 
# Selecionar App -> Selecionar dispositivo -> Gerar
# Vai gerar uma senha aleatória, você vai usar essa senha nessa aplicação
# É só informar o seu e-mail em (login) e senha em (password)

#Adicionar configurações do servidor (Gmail)
app.smtp_config(smtpserver='smtp.gmail.com', smtpport=465, login='@gmail.com', password='')
#Informar o destinatário
app.send_email(toaddr='')


Se quiser usar o PANDAS para enviar o e-mail para uma lista de pessoas, você pode usar o seguinte exemplo:

In [ ]:
#Iniciar o programa informando quem vai mandar o e-mail e qual é o assunto
app = SendEmail(fromaddr='@gmail.com', subject='Muito obrigado!')
#Carregar arquivo Excel
data = app.read_base(file='base.xlsx')
#Adicionar configurações do servidor (Gmail)
app.smtp_config(smtpserver='smtp.gmail.com', smtpport=465, login='@gmail.com', password='')

#Loop principal
for index, row in data.iterrows():
  print(f"Enviando para: {row['NOME']}")
  #Converter dados em string
  data = row.apply(str)
  app.add_mensage_html(htmlfile='mensagem.html', encoding='UTF-8', data=data)
  #Informar o destinatário
  app.send_email(toaddr=row['DESTINATARIO'])
